In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Weaviate
from langchain.document_loaders import TextLoader

In [2]:
loader = TextLoader("./sotu.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/Users/Ryan/github/weaviate/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
db = Weaviate.from_documents(docs, embeddings, weaviate_url="http://localhost:8080", by_text=False)

In [4]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

In [5]:
docs

[Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': './sotu.txt'}),
 Document(page_content='As Frances Haugen, who is here with us tonight, has shown, we must hold social media platforms accountable for the natio

In [6]:
docs = db.similarity_search_with_score(query, by_text=False)
docs[0]

(Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'_additional': {'vector': [0.006040519, 0.113381244, 0.0010239102, -0.0091875, -0.016507884, 0.0025089795, -0.038824633, 0.006660622, -0.01766236, -0.022744674, 0.0404588

# https://python.langchain.com/docs/integrations/vectorstores/weaviate#question-answering-with-sources

In [3]:
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQAWithSourcesChain
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

In [4]:
model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-1b7")
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-1b7")
gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_new_tokens=200)

llm = HuggingFacePipeline(pipeline=gen)

In [ ]:
# llm = HuggingFacePipeline.from_model_id(
#     model_id="bigscience/bloom-1b7",
#     task="text-generation",
#     model_kwargs={
#         # "temperature": 0, 
#         # "do_sample":True,
#         # "max_length": 300,
#         "min_length":50
#         },
# )

In [ ]:
with open("./sotu.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

In [ ]:
docsearch = Weaviate.from_texts(
    texts,
    embeddings,
    weaviate_url="http://localhost:8080",
    by_text=False,
    metadatas=[{"source": f"{i}-pl"} for i in range(len(texts))],
)

In [ ]:
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [ ]:
chain(
    {"question": "What did the president say about Justice Breyer"},
    return_only_outputs=True,
)